In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
import random
import time

### Fetching & Storing the data

In [2]:
headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
          'Accept': '*/*','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8','Connection': 'keep-alive',
          'Content-Length':'34','Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8','Host': 'main.sci.gov.in','Referer': 'https://main.sci.gov.in/case-status'}

In [4]:
#Creating Empty Lists
act=[]
diaryno=[]
caseno=[]
listedon=[]
status=[]
disp=[]
pending=[]
category=[]
petitioner=[]
respondent=[]
petadvocate=[]
resadvocate=[]
usection=[]

In [ ]:
s=requests.session()  # Creating Session
payload={}            #Creating Empty Payload Dictionary
payload['d_yr']=2000  #Initializing d_yr to year 2000
starttime=time.time() # to calculate the time required
print(time.ctime())   #printing the starting time
for payload['d_yr'] in range(2000,2021): #looping till we get the case diary for all the required years
  for i in range(1,101): #Looping till we get the all the case diary we want for each year
    payload['d_no']=i    #Initializing d_no with i
    payload['ansCaptcha']=random.randint(1111,9999) # generating 4 digit random number to bypass captcha
    try:
         #posting the form data in try block
      response=s.post('https://main.sci.gov.in/php/case_status/case_status_process.php',data=payload,headers=headers)
      soup=BeautifulSoup(response.text,'html.parser') #creating soup object of recieved data
    except:
        #catching exceptions in except block
        print('Request Timeout Requesting Again')
        continue
    tab = soup.find('div' , class_='panel-collapse collapse in') #searching the required div using class attributes
    if tab is None: # handling the possibility of None return
      tabdata=soup.find_all('td')
    else:
      tabdata=tab.find_all('td')
    tabledata=[]
    for td in tabdata: #creating a list of td block in the tabdata
      tabledata.append(td.text.strip())
    for i in range(0,len(tabledata),2):#iterating over list to find & append data to respective lists
      info=tabledata[i]
      if ('Act' in info):
        if tabledata[i+1] == '':
          act.append(np.nan) #appending nan value if there are no data for that columns
        else:
          act.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Case No' in info):
        if tabledata[i+1] == '':
          caseno.append(np.nan)
        else:
          caseno.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Category' in info):
        if tabledata[i+1] == '':
          category.append(np.nan)
        else:
          category.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Diary' in info):
        if tabledata[i+1] == '':
          diaryno.append(np.nan)
        else:
          diaryno.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Pet.' in info):
        if tabledata[i+1] == '':
          petadvocate.append(np.nan)
        else:
          petadvocate.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Petitioner' in info):
        if tabledata[i+1] == '':
          petitioner.append(np.nan)
        else:
          petitioner.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Present' in info):
        if tabledata[i+1] == '':
          listedon.append(np.nan)
        else:
          listedon.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Resp.' in info):
        if tabledata[i+1] == '':
          resadvocate.append(np.nan)
        else:
          resadvocate.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Respondent' in info):
        if tabledata[i+1] == '':
          respondent.append(np.nan)
        else:
          respondent.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Status/Stage' in info):
        if tabledata[i+1] == '':
          status.append(np.nan)
        else:
          status.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('Disp.' in info):
        if tabledata[i+1] == '':
          disp.append(np.nan)
        else:
          disp.append(tabledata[i+1].replace('\xa0','').strip())
      elif ('U/Section' in info):
        if tabledata[i+1] == '':
          usection.append(np.nan)
        else:usection.append(tabledata[i+1].replace('\xa0','').strip())
  print(f"Fetching done of year {payload['d_yr']}")    #printing a feedback msg after completing fetching of that year
  payload['d_yr']=payload['d_yr']+1 #updating the d_yr
print('Time required for execution:',time.time()-starttime)

Tue Mar 30 22:25:02 2021
Request Timeout Requesting Again
Request Timeout Requesting Again
Request Timeout Requesting Again
Request Timeout Requesting Again


### Creating DataFrame & CSV

In [16]:
df = {'Diary No':diaryno,'Case No':caseno,'Present/Last Listed On':listedon,'Status/Stage':status,
      'Disp.Type':disp,'Category':category,'Act':act,'Petitioner(s)':petitioner,
      'Respondent(s)':respondent,'Pet. Advocate(s)':petadvocate,'Resp. Advocate(s)':resadvocate,'U/Section':usection}


In [17]:
dataset = pd.DataFrame.from_dict(data = df,orient='index')
dataset=dataset.transpose()
dataset.to_csv('supremecourtcasediary.csv', index=False, encoding='utf-8')

In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2092 entries, 0 to 2091
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Diary No                2092 non-null   object
 1   Case No                 2092 non-null   object
 2   Present/Last Listed On  607 non-null    object
 3   Status/Stage            2063 non-null   object
 4   Disp.Type               1854 non-null   object
 5   Category                2033 non-null   object
 6   Act                     34 non-null     object
 7   Petitioner(s)           2092 non-null   object
 8   Respondent(s)           2092 non-null   object
 9   Pet. Advocate(s)        1895 non-null   object
 10  Resp. Advocate(s)       780 non-null    object
 11  U/Section               35 non-null     object
dtypes: object(12)
memory usage: 196.2+ KB


In [19]:
casedata=pd.read_csv('supremecourtcasediary.csv')

In [32]:
casedata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2092 entries, 0 to 2091
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Diary No                2092 non-null   object
 1   Case No                 2092 non-null   object
 2   Present/Last Listed On  607 non-null    object
 3   Status/Stage            2063 non-null   object
 4   Disp.Type               1854 non-null   object
 5   Category                2033 non-null   object
 6   Act                     34 non-null     object
 7   Petitioner(s)           2092 non-null   object
 8   Respondent(s)           2092 non-null   object
 9   Pet. Advocate(s)        1895 non-null   object
 10  Resp. Advocate(s)       780 non-null    object
 11  U/Section               35 non-null     object
dtypes: object(12)
memory usage: 212.5+ KB
